In [1]:
%%clingo 0 
#const n=9.
time(1..n).

% + EarliestDeparture
% + LatestArrival


% environment #4
cell((1,1),72). cell((2,1),2064). cell((3,1),72). cell((4,1),2064).
cell((1,2),16386). cell((2,2),34864). cell((3,2),32800). cell((4,2),32800).
cell((1,3),72). cell((2,3),37408). cell((3,3),32800). cell((4,3),32800).
cell((1,4),16386). cell((2,4),17411). cell((3,4),5633). cell((4,4),4608).

start(agent(1),(4,2),1,dir(n)). 
end(agent(1),(1,2),10).
start(agent(2),(4,3),1,dir(n)). 
end(agent(2),(1,3),10).

%_________________Encode Tracks____________________

%straight tracks
track(1025, (1, 3)). track(32800, (0, 2)). track(4608, (3, 2)). track(16386, (2, 1)). track(72, (0, 1)). track(2064, (0, 3)).

%simple switch
track(3089, (3, 1)). track(3089, (3, 0)). track(37408, (3, 2)). track(37408, (0, 2)). track(17411, (2, 3)). track(17411, (1, 3)).
track(32872, (0, 1)). track(32872, (0, 2)). track(49186, (0, 2)). track(49186, (2, 1)). track(1097, (1, 3)). track(1097, (0, 1)).
track(34864, (3, 0)). track(34864, (0, 2)). track(5633, (3, 1)). track(5633, (3, 2)).

%diamond crossing
track(33825, (0, 2)). track(33825, (1, 3)).

%diamond plus crossing
track(38433, (0, 2)). track(38433, (1, 3)). track(38433, (3, 2)). track(50211, (0, 2)). track(50211, (1, 3)). track(50211, (2, 1)).
track(33897, (0, 2)). track(33897, (1, 3)). track(33897, (0, 1)). track(35889, (0, 2)). track(35889, (1, 3)). track(35889, (0, 3)).
track(38505, (0, 2)). track(38505, (1, 3)). track(38505, (3, 2)). track(38505, (0, 1)).
track(52275, (0, 2)). track(52275, (1, 3)). track(52275, (2, 1)). track(52275, (0, 3)).

%double curve crossing
track(20994, (1, 2)). track(20994, (2, 3)). track(16458, (0, 1)). track(16458, (1, 2)). 
track(2136, (0, 1)). track(2136, (3, 0)). track(6672, (2, 3)). track(6672, (3, 0)).

%track end
track(8192, (2, end)). track(4, (1, end)). track(128, (0, end)). track(256, (3, end)).

%_________________Generate__________________________

track(N, (X, Y)) :- track(N, (Y, X)).

moveDir(0, (0, 1)). %go north 
moveDir(1, (1, 0)). %go east
moveDir(2, (0, -1)). %go south
moveDir(3, (-1, 0)). %go west

revDir(0, 2). revDir(2, 0). revDir(1, 3). revDir(3, 1).

dir2int(n, 0).
dir2int(e, 1).
dir2int(s, 2).
dir2int(w, 3).

%define tracks where we can turn left depending on the incoming direction
move(Type, DirOut, move_left) :- track(Type, (DirIn, DirOut)), DirOut = DirIn + 1, cell((_,_),Type).
move(Type, 3, move_left) :- track(Type, (0, 3)), cell((_,_),Type).

%define tracks where we move forward depending on the incoming direction
move(Type, DirOut, move_forward) :- track(Type, (DirIn, DirOut)), |DirOut - DirIn| = 2, cell((_,_),Type).

%define tracks where we can turn right depending on the incoming direction
move(Type, DirOut, move_right) :- track(Type, (DirIn, DirOut)), DirOut = DirIn - 1, cell((_,_),Type).
move(Type, 0, move_right) :- track(Type, (3, 0)), cell((_,_),Type).

agent(A) :- start(agent(A), _, _, _).
action_type(move_forward). action_type(wait). action_type(move_left). action_type(move_right).
    
{action(agent(A), Action, Timestamp)} :- start(agent(A),(Y,X), _, dir(D)), action_type(Action), time(Timestamp).
    
%action(agent(A), wait, 0) :- agent(A).
%one action for each agent and each timestamp
:- time(T), agent(A), not 1 {action(agent(A), _, T)} 1.

% cant do anything other than wait before earliert departure time
:- time(T), start(agent(A), _, EarliestDeparture, _), current_state(agent(A), (Y,X), T, DirOut), EarliestDeparture > T, not action(agent(A), wait, T).
     
current_state(agent(A), (Y,X), 0, Dir) :- start(agent(A),(Y,X),_,dir(D)), dir2int(D, Dir).  
    
current_state(agent(A), (Y + DY, X+DX), T + 1, DirOut2) :- current_state(agent(A), (Y,X), T, DirOut),
    action(agent(A), Action, T + 1), cell((Y,X), Type), cell((Y + DY, X+DX), Type2),
    track(Type, (_, DirOut)), track(Type2, (DirIn,DirOut2)), move(Type, DirOut, Action),
    revDir(DirIn, DirOut), moveDir(DirOut, (DY,DX)).

current_state(agent(A), (Y,X), T + 1, Dir) :- current_state(agent(A), (Y,X), T, Dir), action(agent(A), wait, T + 1).
    
:- action(agent(A), _, T), not current_state(agent(A), _, T, _).

%vertex conflict
:-current_state(agent(A1), (Y,X), T, _), current_state(agent(A2), (Y,X), T, _), A1 < A2.

%also include edge conflict later 
    
%agents have to reach destination
:- end(agent(A), (Y,X), LatestArrival), not current_state(agent(A), (Y,X), _, _). 

% Deadline
:- end(agent(A), (Y,X), LatestArrival), current_state(agent(A), (Y,X), T, _), T > LatestArrival.
    
#show action/3.
%#show current_state/4.
%#show move/3.
%#show track/2.

clingo version 5.6.2
Reading from stdin


Solving...
Answer: 1
action(agent(1),move_forward,1) action(agent(1),move_forward,2) action(agent(1),move_forward,5) action(agent(1),move_forward,8) action(agent(1),move_left,3) action(agent(1),move_left,4) action(agent(1),move_left,6) action(agent(1),move_left,7) action(agent(1),move_left,9) action(agent(2),move_forward,1) action(agent(2),move_forward,4) action(agent(2),move_forward,8) action(agent(2),wait,9) action(agent(2),move_left,2) action(agent(2),move_left,3) action(agent(2),move_left,5) action(agent(2),move_left,6) action(agent(2),move_right,7)
Answer: 2
action(agent(1),move_forward,1) action(agent(1),move_forward,2) action(agent(1),move_forward,5) action(agent(1),move_left,3) action(agent(1),move_left,4) action(agent(1),move_left,6) action(agent(1),move_left,7) action(agent(1),move_left,9) action(agent(1),move_right,8) action(agent(2),move_forward,1) action(agent(2),move_forward,4) action(agent(2),move_forward,8) action(agent(2),wait,9) action(agent(2),move_left,2) action(age